In [2]:
import requests
import xmltodict
import json # json 처리
import pymysql
import time
import random

In [3]:
# !chcp 65001

Active code page: 65001


In [3]:
# !conda info --envs 환경

# conda environments:
#
base                     C:\ProgramData\Miniconda3
ssac-one              *  C:\ProgramData\Miniconda3\envs\ssac-one
                         C:\Users\kjk66\anaconda3



# 코로나 해외 발병현황(20200101 - 20201231)

In [32]:
# 코로나 OpenAPI 연계
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19NatInfStateJson'
api_key = '3fFyL9wc7wsGOpUkhxQchp0AZBH8PlyNELGTdLSBGeXMNgaPRcHbWZ%2FsKlNnnw4q67omLLYJM34Yu1osH58d9w%3D%3D'

qs = 'ServiceKey={0}&startCreateDt={1}&endCreateDt={2}'.format(api_key, '20200101', '20201231') 

response = requests.get(url + "?" + qs)
response_dict = xmltodict.parse(response.content) # xml -> ordereddict
response_json = json.dumps(response_dict, ensure_ascii=False) # ordereddict -> json-string
response_dict2 = json.loads(response_json) # json-stirng -> general dict
values = response_dict2['response']['body']['items']['item']

In [33]:
# 데이터 확인

print(type(values))
print(len(values))
values[0]

<class 'list'>
55471


{'areaNm': '기타',
 'areaNmCn': '其他',
 'areaNmEn': 'Others',
 'createDt': '2020-12-31 10:14:39.677',
 'natDeathCnt': '3785',
 'natDeathRate': '1.1921109655',
 'natDefCnt': '317504',
 'nationNm': '기타지역 영토',
 'nationNmCn': '其他',
 'nationNmEn': 'Others',
 'seq': '67239',
 'stdDay': '2020년 12월 31일 09시',
 'updateDt': 'null'}

In [ ]:
# 응답메시지 설명
global_covid_desc = {'areaNm' : '지역명', 
                     'areaNmCn' : '지역명(중문)', 
                     'areaNmEn' : '지역명(영문)', 
                     'createDt' : '등록일시분초', 
                     'natDeathCnt' : '국가별 사망자 수', 
                     'natDeathRate' : '확진률 대비 사망률', 
                     'natDefCnt' : '국가별 확진자 수', 
                     'nationNm' : '국가명', 
                     'nationNmCn' : '국가명(중문)', 
                     'nationNmEn' : '지역명(영문)', 
                     'seq' : '게시글번호(고유)', 
                     'stdDay' : '기준일시', 
                     'updateDt' : '수정일시분초'}

In [ ]:
# 코로나 해외발생현황 테이블 생성 (mysql 작업)
"""
create table global_def_cnt(
    seq int primary key, 
	area varchar(30), 
    area_cn varchar(30),
    area_en varchar(30), 
    creat_dt datetime not null, 
    nat_death_cnt int, 
    nat_death_rate float, 
    nat_def_cnt int, 
    nat varchar(30), 
    nat_cn varchar(30), 
    nat_en varchar(30), 
    stdday date not null, 
    update_dt datetime);  
"""

In [34]:
# api 파라미터별 컬럼 매핑

columns = ['area', 'area_cn', 'area_en', 'creat_dt', 'nat_death_cnt', 'nat_death_rate', 'nat_def_cnt', 'nat', 'nat_cn', 'nat_en', 'seq', 'stdday', 'update_dt']

keys = ['areaNm', 'areaNmCn', 'areaNmEn', 'createDt', 'natDeathCnt', 'natDeathRate', 'natDefCnt', 'nationNm',	'nationNmCn', 'nationNmEn', 'seq', 'stdDay', 'updateDt']

key_column_map = { key: column for key, column in zip(keys, columns) }
print( key_column_map )
column_key_map = { column: key for key, column in zip(keys, columns) }
print( column_key_map )

len(keys), len(columns)



{'areaNm': 'area', 'areaNmCn': 'area_cn', 'areaNmEn': 'area_en', 'createDt': 'creat_dt', 'natDeathCnt': 'nat_death_cnt', 'natDeathRate': 'nat_death_rate', 'natDefCnt': 'nat_def_cnt', 'nationNm': 'nat', 'nationNmCn': 'nat_cn', 'nationNmEn': 'nat_en', 'seq': 'seq', 'stdDay': 'stdday', 'updateDt': 'update_dt'}
{'area': 'areaNm', 'area_cn': 'areaNmCn', 'area_en': 'areaNmEn', 'creat_dt': 'createDt', 'nat_death_cnt': 'natDeathCnt', 'nat_death_rate': 'natDeathRate', 'nat_def_cnt': 'natDefCnt', 'nat': 'nationNm', 'nat_cn': 'nationNmCn', 'nat_en': 'nationNmEn', 'seq': 'seq', 'stdday': 'stdDay', 'update_dt': 'updateDt'}


(13, 13)

In [35]:
import pymysql

connection_info = { 'host': 'localhost', 'user': 'root', 'password': 'Pa$$w0rd', 'db': 'prj1', 'charset': 'utf8' }
        
conn = pymysql.connect(**connection_info)
cursor = conn.cursor()

sql = 'DELETE FROM global_def_cnt'
cursor.execute(sql)

sql = 'INSERT INTO values VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
cursor.executemany(sql, values) # excutemany 여러개의 데이터를 한번에 일괄 삽입
# insert into tbl_name (columnlist) values (), (), (), ()
conn.commit()

conn.close()

TypeError: not enough arguments for format string

# 미세먼지 데이터

In [6]:
# https://data.seoul.go.kr/dataList/OA-2218/S/1/datasetView.do
# 서울시 일별 평균 대기오염도 정보 (2019~2020)
# 파일데이터(csv)

In [52]:
import csv
import re

dust_info = []
with open('D:\ssac-one\demo-repo\data\일별평균대기오염도.csv') as f:
    reader = csv.reader(f)
    header = next(reader)
    for line in reader:        
        for i in range(2, 8):
            line[i] = 0 if line[i] == '' else line[i]
        dust_info.append(line)

In [3]:
# 미세먼지 관측정보 테이블 생성 (mysql 작업)
"""
create table obs_dust(
	obs_dt date not null,
    stn_nm varchar(20) primary key,
    no2 double default 0,
    o3 double default 0,
    co2 double default 0,
    so2 double default 0,
    pm10 int default 0,
    pm25 int default 0);
"""

'\ncreate table obs_dust(\n\tobs_dt date not null,\n    stn_nm varchar(20) primary key,\n    no2 double,\n    o3 double,\n    co2 double,\n    so2 double,\n    pm10 int,\n    pm25 int);\n'

In [53]:
dust_info[8]

['20190101', '관악산', '0.011', '0.024', 0, '0.004', '31', '18']

In [54]:
connection_info = { 'host': 'localhost', 'user': 'root', 'password': 'Pa$$w0rd', 'db': 'prj1', 'charset': 'utf8' }
        
conn = pymysql.connect(**connection_info)
cursor = conn.cursor()

sql = 'INSERT INTO obs_dust VALUES(%s, %s, %s, %s, %s, %s, %s, %s)'
cursor.executemany(sql, dust_info) 
conn.commit()

conn.close()

IntegrityError: (1062, "Duplicate entry '강남구' for key 'obs_dust.PRIMARY'")

In [10]:
print(dust_info[10])
print(dust_info[10][4])

['20190101', '구로구', '0.028', '0.016', '0.4', '0.006', '34', '18']
0.4


In [55]:
print(dust_info[8])
print(dust_info[8][4])

['20190101', '관악산', '0.011', '0.024', 0, '0.004', '31', '18']
0
